In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Installing libraries

In [2]:
!pip install textattack
!pip install -U gensim==4.0.0
!pip install python-Levenshtein

Requirement already up-to-date: gensim==4.0.0 in /usr/local/lib/python3.7/dist-packages (4.0.0)


In [3]:
# import gensim.downloader as api
# corpus = api.load('text8')
# from gensim.models.fasttext import FastText
# model1 = FastText(corpus, sample = 0, sg = 1)

In [4]:
# import gensim.downloader as api
# corpus = api.load('text8')
# from gensim.models.word2vec import Word2Vec
# model2 = Word2Vec(corpus, sample = 0, sg = 1)

# Loading FastText, Word2Vec and Glove

In [5]:
from gensim.test.utils import get_tmpfile
from gensim.models.fasttext import FastText
#fname = get_tmpfile("/models/fasttext.model") 
model1 = FastText.load("/content/drive/MyDrive/models/fasttext.model") # Change this location

In [6]:
from gensim.test.utils import get_tmpfile
from gensim.models.word2vec import Word2Vec
#fname1 = get_tmpfile("/models/word2vec.model") 
model2 = Word2Vec.load("/content/drive/MyDrive/models/word2vec.model") # Change this location

In [7]:
from gensim.models import KeyedVectors
model3=KeyedVectors.load('/content/drive/MyDrive/models/gloveKeyVectors.kv')

Example demonstration

In [8]:
print(model1.wv.most_similar('cat'))
print(model2.wv.most_similar('cat'))
print(model3.most_similar('cat'))

[('bobcat', 0.759185791015625), ('dog', 0.7208123803138733), ('foxglove', 0.7110776901245117), ('pig', 0.7088143229484558), ('goose', 0.7069101333618164), ('cats', 0.702387273311615), ('beak', 0.7008379697799683), ('catfish', 0.6987316608428955), ('foxhound', 0.696578860282898), ('catnip', 0.6960986852645874)]
[('dog', 0.7193913459777832), ('squirrel', 0.696162760257721), ('leopardus', 0.6945451498031616), ('prionailurus', 0.688106894493103), ('albino', 0.6880608201026917), ('bird', 0.6836060285568237), ('eared', 0.6803243160247803), ('rhinoceros', 0.6769237518310547), ('hyena', 0.6756084561347961), ('badger', 0.6752869486808777)]
[('cats', 0.7006820440292358), ('dog', 0.698471188545227), ('dogs', 0.6442825198173523), ('tervueren', 0.619421660900116), ('laekenois', 0.5972026586532593), ('lica', 0.5956287980079651), ('feral', 0.5899650454521179), ('goat', 0.5861616134643555), ('fish', 0.5746133327484131), ('malinois', 0.5737614631652832)]


In [9]:
if 'cat' in model3.key_to_index:
  print('present')

present


# Base-uncased Ag-news

In [10]:
# Import the model
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-ag-news")
tokenizer = AutoTokenizer("textattack/bert-base-uncased-ag-news")
# Change these

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# Framing fasttext attack

In [11]:
from textattack.transformations import WordSwap

class Swapper(WordSwap): # For fasttext
    """ Transforms an input by replacing any word with its most similar counterpart
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        for i in range(10):
            if word.lower() in model1.wv.most_similar(word)[i][0].lower():
                continue # Don't return the exact word
            elif word.isupper():
                return [model1.wv.most_similar(word)[i][0].upper()] # Preserving case
            elif word[0].isupper():
                return [model1.wv.most_similar(word)[i][0].capitalize()] # Preserving Capitalization in words
            else:
                return [model1.wv.most_similar(word)[i][0]]
            
        return [model1.wv.most_similar(word)[0][0]]

In [12]:
def printexample(attack):
  print(attack)
  from collections import deque

  import textattack
  import tqdm
  import time
  from IPython.display import display, HTML
  num_examples = 25
  num_remaining_attacks = num_examples
  pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

  worklist = deque(range(0, num_examples))
  worklist_tail = worklist[-1]

  attack_log_manager = textattack.loggers.AttackLogManager()

  load_time = time.time()
        
  num_results = 0
  num_failures = 0
  num_successes = 0
  for result in attack.attack_dataset(dataset, indices=worklist):
      result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
      display(HTML(result_html_str))
      attack_log_manager.log_result(result)
            
      if not isinstance(result, textattack.attack_results.SkippedAttackResult):
          pbar.update(1)
      else:
          worklist_tail += 1
          pbar.update(1)
          worklist.append(worklist_tail)

      num_results += 1

      if (type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult):
          num_successes += 1
                
      if type(result) == textattack.attack_results.FailedAttackResult:
          num_failures += 1
      pbar.set_description(
          "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
      )

  pbar.close()

  attack_log_manager.enable_stdout()
  attack_log_manager.log_summary()
  attack_log_manager.flush()
        
  textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

In [13]:
from textattack.search_methods import GreedySearch
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedySearch()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [14]:
print(attack)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [15]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:02<00:55,  2.32s/it]

[Succeeded / Failed / Total] 2 / 0 / 2:   8%|▊         | 2/25 [00:13<02:35,  6.74s/it]

[Succeeded / Failed / Total] 2 / 1 / 3:  12%|█▏        | 3/25 [00:30<03:45, 10.27s/it]

...

[Succeeded / Failed / Total] 12 / 11 / 24:  96%|█████████▌| 24/25 [09:50<00:24, 24.61s/it]

[Succeeded / Failed / Total] 13 / 11 / 25: 100%|██████████| 25/25 [09:51<00:00, 23.68s/it]

[Succeeded / Failed / Total] 13 / 12 / 26: : 26it [09:59, 23.05s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 13     |
| Number of failed attacks:     | 12     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 96.15% |
| Accuracy under attack:        | 46.15% |
| Attack success rate:          | 52.0%  |
| Average perturbed word %:     | 19.98% |
| Average num. words per input: | 45.54  |
| Avg num queries:              | 558.24 |
+-------------------------------+--------+


textattack: Attack time: 599.1855063438416s


# Framing word2vec attack

In [16]:
from textattack.transformations import WordSwap

class Swapper_(WordSwap): # For word2vec
    """ Transforms an input by replacing any word with its most similar counterpart
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        """ Returns 'banana', no matter what 'word' was originally.

            Returns a list with one item, since `_get_replacement_words` is intended to
                return a list of candidate replacement words.
        """
        if word in model2.wv.key_to_index:
            for i in range(10):
                if word.lower() in model2.wv.most_similar(word)[i][0].lower():
                    continue
                elif word.isupper():
                    return [model2.wv.most_similar(word)[i][0].upper()]
                elif word[0].isupper():
                    return [model2.wv.most_similar(word)[i][0].capitalize()]
                else:
                    return [model2.wv.most_similar(word)[i][0]]
        else:
            return [word]

In [17]:
from textattack.search_methods import GreedySearch
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedySearch()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [18]:
print(attack)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [19]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:01<00:44,  1.85s/it]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:09<01:50,  4.80s/it]

[Succeeded / Failed / Total] 0 / 3 / 3:  12%|█▏        | 3/25 [00:15<01:54,  5.21s/it]

...

[Succeeded / Failed / Total] 2 / 21 / 24:  96%|█████████▌| 24/25 [04:01<00:10, 10.06s/it]

[Succeeded / Failed / Total] 2 / 22 / 25: 100%|██████████| 25/25 [04:04<00:00,  9.78s/it]

[Succeeded / Failed / Total] 2 / 23 / 26: : 26it [04:08,  9.56s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 23     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 96.15% |
| Accuracy under attack:        | 88.46% |
| Attack success rate:          | 8.0%   |
| Average perturbed word %:     | 3.65%  |
| Average num. words per input: | 45.54  |
| Avg num queries:              | 246.12 |
+-------------------------------+--------+


textattack: Attack time: 248.6719183921814s


# Forming Glove attack

In [20]:
from textattack.transformations import WordSwap

class Swapper__(WordSwap): # For glove
    """ Transforms an input by replacing any word with its most similar counterpart
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        """ Returns 'banana', no matter what 'word' was originally.

            Returns a list with one item, since `_get_replacement_words` is intended to
                return a list of candidate replacement words.
        """
        if word in model3.key_to_index:
            for i in range(10):
                if word.lower() in model3.most_similar(word)[i][0].lower():
                    continue
                elif word.isupper():
                    return [model3.most_similar(word)[i][0].upper()]
                elif word[0].isupper():
                    return [model3.most_similar(word)[i][0].capitalize()]
                else:
                    return [model3.most_similar(word)[i][0]]
        else:
            return [word]

In [21]:
from textattack.search_methods import GreedySearch
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedySearch()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [22]:
print(attack)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [23]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:02<01:10,  2.92s/it]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:16<03:07,  8.13s/it]

[Succeeded / Failed / Total] 0 / 3 / 3:  12%|█▏        | 3/25 [00:25<03:10,  8.66s/it]

...

[Succeeded / Failed / Total] 3 / 20 / 24:  96%|█████████▌| 24/25 [06:39<00:16, 16.65s/it]

[Succeeded / Failed / Total] 3 / 21 / 25: 100%|██████████| 25/25 [06:44<00:00, 16.19s/it]

[Succeeded / Failed / Total] 3 / 22 / 26: : 26it [06:51, 15.82s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 3      |
| Number of failed attacks:     | 22     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 96.15% |
| Accuracy under attack:        | 84.62% |
| Attack success rate:          | 12.0%  |
| Average perturbed word %:     | 8.31%  |
| Average num. words per input: | 45.54  |
| Avg num queries:              | 253.28 |
+-------------------------------+--------+


textattack: Attack time: 411.4012770652771s


In [24]:
from textattack.search_methods import GreedyWordSwapWIR
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [25]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [26]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:01<00:39,  1.64s/it]

[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:05<01:05,  2.85s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:08<01:04,  2.93s/it]

...

[Succeeded / Failed / Total] 8 / 15 / 24:  96%|█████████▌| 24/25 [01:12<00:03,  3.04s/it]

[Succeeded / Failed / Total] 9 / 15 / 25: 100%|██████████| 25/25 [01:14<00:00,  2.97s/it]

[Succeeded / Failed / Total] 9 / 16 / 26: : 26it [01:16,  2.93s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 9      |
| Number of failed attacks:     | 16     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 96.15% |
| Accuracy under attack:        | 61.54% |
| Attack success rate:          | 36.0%  |
| Average perturbed word %:     | 19.19% |
| Average num. words per input: | 45.54  |
| Avg num queries:              | 72.36  |
+-------------------------------+--------+


textattack: Attack time: 76.18589544296265s


In [27]:
from textattack.search_methods import GreedyWordSwapWIR
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [28]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [29]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:01<00:32,  1.36s/it]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:04<00:47,  2.09s/it]

[Succeeded / Failed / Total] 0 / 3 / 3:  12%|█▏        | 3/25 [00:06<00:48,  2.19s/it]

...

[Succeeded / Failed / Total] 2 / 21 / 24:  96%|█████████▌| 24/25 [00:58<00:02,  2.46s/it]

[Succeeded / Failed / Total] 2 / 22 / 25: 100%|██████████| 25/25 [01:00<00:00,  2.44s/it]

[Succeeded / Failed / Total] 2 / 23 / 26: : 26it [01:02,  2.41s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 23     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 96.15% |
| Accuracy under attack:        | 88.46% |
| Attack success rate:          | 8.0%   |
| Average perturbed word %:     | 7.29%  |
| Average num. words per input: | 45.54  |
| Avg num queries:              | 63.44  |
+-------------------------------+--------+


textattack: Attack time: 62.56433176994324s


In [30]:
from textattack.search_methods import GreedyWordSwapWIR
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [31]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [32]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:01<00:37,  1.55s/it]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:04<00:57,  2.49s/it]

[Succeeded / Failed / Total] 0 / 3 / 3:  12%|█▏        | 3/25 [00:07<00:57,  2.62s/it]

...

[Succeeded / Failed / Total] 3 / 20 / 24:  96%|█████████▌| 24/25 [01:10<00:02,  2.95s/it]

[Succeeded / Failed / Total] 3 / 21 / 25: 100%|██████████| 25/25 [01:12<00:00,  2.92s/it]

[Succeeded / Failed / Total] 3 / 22 / 26: : 26it [01:15,  2.89s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 3      |
| Number of failed attacks:     | 22     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 96.15% |
| Accuracy under attack:        | 84.62% |
| Attack success rate:          | 12.0%  |
| Average perturbed word %:     | 10.74% |
| Average num. words per input: | 45.54  |
| Avg num queries:              | 63.44  |
+-------------------------------+--------+


textattack: Attack time: 75.07407855987549s


In [33]:
from textattack.search_methods import ParticleSwarmOptimization
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5,post_turn_check=True)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [34]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [35]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:02<01:10,  2.95s/it]

[Succeeded / Failed / Total] 2 / 0 / 2:   8%|▊         | 2/25 [00:57<11:03, 28.85s/it]

[Succeeded / Failed / Total] 2 / 1 / 3:  12%|█▏        | 3/25 [01:49<13:21, 36.41s/it]

...

[Succeeded / Failed / Total] 10 / 13 / 24:  96%|█████████▌| 24/25 [22:20<00:55, 55.84s/it]

[Succeeded / Failed / Total] 11 / 13 / 25: 100%|██████████| 25/25 [22:39<00:00, 54.37s/it]

[Succeeded / Failed / Total] 11 / 14 / 26: : 26it [22:44, 52.48s/it]

+-------------------------------+---------+
| Attack Results                |         |
+-------------------------------+---------+
| Number of successful attacks: | 11      |
| Number of failed attacks:     | 14      |
| Number of skipped attacks:    | 1       |
| Original accuracy:            | 96.15%  |
| Accuracy under attack:        | 53.85%  |
| Attack success rate:          | 44.0%   |
| Average perturbed word %:     | 21.67%  |
| Average num. words per input: | 45.54   |
| Avg num queries:              | 1161.24 |
+-------------------------------+---------+


textattack: Attack time: 1364.5365228652954s


In [36]:
from textattack.search_methods import ParticleSwarmOptimization
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5,post_turn_check=True)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [37]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [38]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:02<01:11,  2.99s/it]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:33<06:21, 16.59s/it]

[Succeeded / Failed / Total] 0 / 3 / 3:  12%|█▏        | 3/25 [00:53<06:35, 17.98s/it]

...

[Succeeded / Failed / Total] 2 / 21 / 24:  96%|█████████▌| 24/25 [10:55<00:27, 27.33s/it]

[Succeeded / Failed / Total] 2 / 22 / 25: 100%|██████████| 25/25 [11:12<00:00, 26.89s/it]

[Succeeded / Failed / Total] 2 / 23 / 26: : 26it [11:19, 26.13s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 23     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 96.15% |
| Accuracy under attack:        | 88.46% |
| Attack success rate:          | 8.0%   |
| Average perturbed word %:     | 3.65%  |
| Average num. words per input: | 45.54  |
| Avg num queries:              | 720.08 |
+-------------------------------+--------+


textattack: Attack time: 679.4515733718872s


In [39]:
from textattack.search_methods import ParticleSwarmOptimization
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5,post_turn_check=True)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [40]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [41]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:10<04:07, 10.32s/it]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:51<09:56, 25.95s/it]

[Succeeded / Failed / Total] 0 / 3 / 3:  12%|█▏        | 3/25 [01:31<11:12, 30.58s/it]

...

[Succeeded / Failed / Total] 3 / 20 / 24:  96%|█████████▌| 24/25 [16:44<00:41, 41.85s/it]

[Succeeded / Failed / Total] 3 / 21 / 25: 100%|██████████| 25/25 [17:06<00:00, 41.05s/it]

[Succeeded / Failed / Total] 3 / 22 / 26: : 26it [17:26, 40.23s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 3      |
| Number of failed attacks:     | 22     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 96.15% |
| Accuracy under attack:        | 84.62% |
| Attack success rate:          | 12.0%  |
| Average perturbed word %:     | 12.23% |
| Average num. words per input: | 45.54  |
| Avg num queries:              | 728.08 |
+-------------------------------+--------+


textattack: Attack time: 1046.0094492435455s


In [42]:
from textattack.search_methods import GreedyWordSwapWIR
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [PartOfSpeech()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [43]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [44]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:01<00:45,  1.88s/it]

[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:05<01:08,  2.97s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [00:09<01:09,  3.14s/it]

...

[Succeeded / Failed / Total] 6 / 17 / 24:  96%|█████████▌| 24/25 [01:18<00:03,  3.26s/it]

[Succeeded / Failed / Total] 7 / 17 / 25: 100%|██████████| 25/25 [01:20<00:00,  3.22s/it]

[Succeeded / Failed / Total] 7 / 18 / 26: : 26it [01:22,  3.17s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 7      |
| Number of failed attacks:     | 18     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 96.15% |
| Accuracy under attack:        | 69.23% |
| Attack success rate:          | 28.0%  |
| Average perturbed word %:     | 22.67% |
| Average num. words per input: | 45.54  |
| Avg num queries:              | 74.16  |
+-------------------------------+--------+


textattack: Attack time: 82.3567042350769s


In [45]:
from textattack.search_methods import GreedyWordSwapWIR
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [PartOfSpeech()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [46]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [47]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:01<00:38,  1.62s/it]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:04<00:52,  2.28s/it]

[Succeeded / Failed / Total] 0 / 3 / 3:  12%|█▏        | 3/25 [00:07<00:53,  2.43s/it]

...

[Succeeded / Failed / Total] 2 / 21 / 24:  96%|█████████▌| 24/25 [01:04<00:02,  2.68s/it]

[Succeeded / Failed / Total] 2 / 22 / 25: 100%|██████████| 25/25 [01:06<00:00,  2.66s/it]

[Succeeded / Failed / Total] 2 / 23 / 26: : 26it [01:08,  2.62s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 23     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 96.15% |
| Accuracy under attack:        | 88.46% |
| Attack success rate:          | 8.0%   |
| Average perturbed word %:     | 16.15% |
| Average num. words per input: | 45.54  |
| Avg num queries:              | 64.72  |
+-------------------------------+--------+


textattack: Attack time: 68.0901427268982s


In [48]:
from textattack.search_methods import GreedyWordSwapWIR
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [PartOfSpeech()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [49]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [50]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:01<00:46,  1.94s/it]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:05<01:05,  2.85s/it]

[Succeeded / Failed / Total] 0 / 3 / 3:  12%|█▏        | 3/25 [00:09<01:07,  3.08s/it]

...

[Succeeded / Failed / Total] 3 / 20 / 24:  96%|█████████▌| 24/25 [01:21<00:03,  3.40s/it]

[Succeeded / Failed / Total] 3 / 21 / 25: 100%|██████████| 25/25 [01:24<00:00,  3.36s/it]

[Succeeded / Failed / Total] 3 / 22 / 26: : 26it [01:26,  3.31s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 3      |
| Number of failed attacks:     | 22     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 96.15% |
| Accuracy under attack:        | 84.62% |
| Attack success rate:          | 12.0%  |
| Average perturbed word %:     | 17.34% |
| Average num. words per input: | 45.54  |
| Avg num queries:              | 62.12  |
+-------------------------------+--------+


textattack: Attack time: 86.11663842201233s


In [51]:
from textattack.search_methods import ParticleSwarmOptimization
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [PartOfSpeech()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [52]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [53]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:08<03:24,  8.53s/it]

[Succeeded / Failed / Total] 2 / 0 / 2:   8%|▊         | 2/25 [00:56<10:50, 28.28s/it]

[Succeeded / Failed / Total] 2 / 1 / 3:  12%|█▏        | 3/25 [02:17<16:49, 45.90s/it]

...

[Succeeded / Failed / Total] 9 / 14 / 24:  96%|█████████▌| 24/25 [26:25<01:06, 66.05s/it]

[Succeeded / Failed / Total] 10 / 14 / 25: 100%|██████████| 25/25 [26:39<00:00, 63.98s/it]

[Succeeded / Failed / Total] 10 / 15 / 26: : 26it [27:00, 62.34s/it]

+-------------------------------+---------+
| Attack Results                |         |
+-------------------------------+---------+
| Number of successful attacks: | 10      |
| Number of failed attacks:     | 15      |
| Number of skipped attacks:    | 1       |
| Original accuracy:            | 96.15%  |
| Accuracy under attack:        | 57.69%  |
| Attack success rate:          | 40.0%   |
| Average perturbed word %:     | 15.39%  |
| Average num. words per input: | 45.54   |
| Avg num queries:              | 1340.72 |
+-------------------------------+---------+


textattack: Attack time: 1620.7315068244934s


In [54]:
from textattack.search_methods import ParticleSwarmOptimization
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [PartOfSpeech()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [55]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [56]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:16<06:37, 16.57s/it]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:56<10:50, 28.30s/it]

[Succeeded / Failed / Total] 0 / 3 / 3:  12%|█▏        | 3/25 [01:40<12:19, 33.61s/it]

...

[Succeeded / Failed / Total] 2 / 21 / 24:  96%|█████████▌| 24/25 [17:04<00:42, 42.71s/it]

[Succeeded / Failed / Total] 2 / 22 / 25: 100%|██████████| 25/25 [17:35<00:00, 42.23s/it]

[Succeeded / Failed / Total] 2 / 23 / 26: : 26it [17:54, 41.32s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 23     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 96.15% |
| Accuracy under attack:        | 88.46% |
| Attack success rate:          | 8.0%   |
| Average perturbed word %:     | 6.77%  |
| Average num. words per input: | 45.54  |
| Avg num queries:              | 943.36 |
+-------------------------------+--------+


textattack: Attack time: 1074.3697218894958s


In [57]:
from textattack.search_methods import ParticleSwarmOptimization
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [PartOfSpeech()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [58]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [59]:
printexample(attack)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:31<12:25, 31.06s/it]

[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [01:41<19:32, 50.96s/it]

[Succeeded / Failed / Total] 1 / 2 / 3:  12%|█▏        | 3/25 [03:01<22:09, 60.44s/it]

...

[Succeeded / Failed / Total] 4 / 19 / 24:  96%|█████████▌| 24/25 [27:20<01:08, 68.36s/it]

[Succeeded / Failed / Total] 4 / 20 / 25: 100%|██████████| 25/25 [28:03<00:00, 67.34s/it]

[Succeeded / Failed / Total] 4 / 21 / 26: : 26it [28:35, 65.98s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 4      |
| Number of failed attacks:     | 21     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 96.15% |
| Accuracy under attack:        | 80.77% |
| Attack success rate:          | 16.0%  |
| Average perturbed word %:     | 17.43% |
| Average num. words per input: | 45.54  |
| Avg num queries:              | 795.04 |
+-------------------------------+--------+


textattack: Attack time: 1715.4736459255219s
